Converting smiles to .pdbqt format for autodock using RDKit

In [1]:
! pip install rdkit-pypi openbabel


  Using cached rdkit_pypi-2022.9.5-cp39-cp39-win_amd64.whl.metadata (4.0 kB)
  Using cached openbabel-3.1.1.1.tar.gz (82 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached rdkit_pypi-2022.9.5-cp39-cp39-win_amd64.whl (20.5 MB)
  Running setup.py clean for openbabel
Failed to build openbabel


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [66 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Guessing Open Babel location:
      - include_dirs: ['C:\\Users\\saman\\Miniconda3\\envs\\bioenv\\include', 'C:\\Users\\saman\\Miniconda3\\envs\\bioenv\\Include', '/usr/local/include/openbabel3']
      - library_dirs: ['C:\\Users\\saman\\Miniconda3\\envs\\bioenv\\libs', 'C:\\Users\\saman\\Miniconda3\\envs\\bioenv', 'C:\\Users\\saman\\Miniconda3\\envs\\bioenv\\PCbuild\\amd64', '/usr/local/lib']
      building 'openbabel._openbabel' extension
      swigging openbabel\openbabel-python.i to openbabel\openbabel-python_wrap.cpp
      swig.exe -python -c++ -small -O -templatereduce -naturalvar -IC:\Users\saman\Miniconda3\envs\bioenv\include -IC:\Users\saman\Miniconda3\envs\bioenv\Include -I/usr/local/include/openbabel3 -o openbabel\openbabel-python_wrap.cpp openbabel\openbabel-

In [2]:
import pandas as pd

#Data with SMILES strings
df= pd.read_csv("dna_gyrase_bioactivity_data_curated.csv")
df

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL187677,C[C@H]1CCc2c(N3CCC(O)CC3)c(F)cc3c(=O)c(C(=O)O)...,5.0,active
1,CHEMBL363449,CC1CCc2c(N3CCC(O)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23,10.0,active
2,CHEMBL8,O=C(O)c1cn(C2CC2)c2cc(N3CCNCC3)c(F)cc2c1=O,5.0,active
3,CHEMBL192226,CN1CCN(c2c(F)cc3c(=O)c(C(=O)O)cn(/C=C\F)c3c2F)CC1,72.2,active
4,CHEMBL371124,CN1CCN(c2c(F)cc3c(=O)c(C(=O)O)cn(/C=C/F)c3c2F)CC1,664.0,active
...,...,...,...,...
541,CHEMBL5431772,COc1ccc2nccc(OC[C@H]3CC[C@H](NC(=O)c4cc(F)c(Br...,100000.0,inactive
542,CHEMBL5429864,COc1ccc2nccc(OC[C@H]3CC[C@H](NC(=O)c4ccc(I)c(F...,100000.0,inactive
543,CHEMBL5394250,COc1ccc2nccc(OC[C@@H]3CC[C@@H](NC(=O)c4cc(F)c(...,445.0,active
544,CHEMBL5406927,COc1ccc2nccc(OC[C@@H]3CC[C@@H](NC(=O)c4cc(F)c(...,185.0,active


In [3]:
top_indices = [94, 85, 89, 104, 114, 533, 271, 93, 92, 113]


In [4]:
# Extract top compounds by row index
top_compounds = df.loc[top_indices, ['molecule_chembl_id', 'canonical_smiles']]


In [5]:
top_compounds

,molecule_chembl_id,canonical_smiles
94,CHEMBL2382359,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5c(F)...
85,CHEMBL2382368,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5cc(F...
89,CHEMBL2382364,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5cc(F...
104,CHEMBL2382349,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5c(F)...
114,CHEMBL2382285,COc1ccc2nccc(C(O)CC[C@@H]3CCN(C/C=C/c4cc(F)ccc...
533,CHEMBL5433070,COc1ccc(-c2csc(NC(=S)N[C@@H]3O[C@H](COC(C)=O)[...
271,CHEMBL3589080,O=C(O)CCC(=O)Nc1nc2c(s1)C[C@@H](NC(=O)c1cc(Cl)...
93,CHEMBL2382360,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5ccc(...
92,CHEMBL2382361,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5ccc(...
113,CHEMBL2382287,COc1ccc2nccc([C@@H](O)CC[C@@H]3CCN(C4CC(c5cccc...


In [6]:
# Save to CSV for docking input
top_compounds.to_csv("top_10_SMILES.csv", index=False)


In [8]:
! pip install pandas rdkit 

In [9]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import os

df= pd.read_csv("top_10_SMILES.csv")

os.makedirs("pdbqt_files", exist_ok=True)

#convert smiles to pdbqt
for i, row in top_compounds.iterrows():
    smiles = row['canonical_smiles']
    chembl_id = row['molecule_chembl_id']
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to parse SMILES: {smiles}")
        continue
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol)

# Save the molecule in .mol format
    mol_file= f"{chembl_id}.mol"
    Chem.MolToMolFile(mol, mol_file)

#convert .mol to .pdbqt using Open Babel
    pdbqt_file = f"pdbqt_files/{chembl_id}.pdbqt"
    os.system(f"obabel {mol_file} -O {pdbqt_file} --gen3d")

#clean up the mol file
    os.remove(mol_file)  
    print(f"Converted {chembl_id} to PDBQT format and saved as {pdbqt_file}")

Converted CHEMBL2382359 to PDBQT format and saved as pdbqt_files/CHEMBL2382359.pdbqt
Converted CHEMBL2382368 to PDBQT format and saved as pdbqt_files/CHEMBL2382368.pdbqt
Converted CHEMBL2382364 to PDBQT format and saved as pdbqt_files/CHEMBL2382364.pdbqt
Converted CHEMBL2382349 to PDBQT format and saved as pdbqt_files/CHEMBL2382349.pdbqt
Converted CHEMBL2382285 to PDBQT format and saved as pdbqt_files/CHEMBL2382285.pdbqt
Converted CHEMBL5433070 to PDBQT format and saved as pdbqt_files/CHEMBL5433070.pdbqt
Converted CHEMBL3589080 to PDBQT format and saved as pdbqt_files/CHEMBL3589080.pdbqt
Converted CHEMBL2382360 to PDBQT format and saved as pdbqt_files/CHEMBL2382360.pdbqt
Converted CHEMBL2382361 to PDBQT format and saved as pdbqt_files/CHEMBL2382361.pdbqt
Converted CHEMBL2382287 to PDBQT format and saved as pdbqt_files/CHEMBL2382287.pdbqt
